In [3]:
import os
import cv2
import random
import numpy as np
import zipfile
import os
import pandas as pd
from keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img

In [8]:
zip_path = "./dataset.zip"  # Adjust if your zip name is different
extract_path = "dataset"     # Where to unzip

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [9]:
main_dir = "./dataset/human_detection"

In [10]:
# Load and preprocess your dataset
def read_and_preprocess_image(image_path):
    image = cv2.imread(image_path, 1)
    image = cv2.resize(image, (224, 224))  # Resize to the input size of MobileNetV2
    return image

In [11]:
data = []
labels = []

for class_name in os.listdir(main_dir):
    class_dir = os.path.join(main_dir, class_name)
    for image_file in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_file)
        image = read_and_preprocess_image(image_path)
        data.append(image)
        labels.append(int(class_name))  # Assuming class names are 0 and 1

# Convert labels to one-hot encoding
labels = to_categorical(labels, num_classes=2)

In [12]:
len(data[0])


224

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [14]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

736
185
736
185


In [15]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [16]:
# Model architecture
model = Sequential()
base_model = MobileNetV2(input_shape=(224, 224, 3))
model.add(base_model)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [42]:
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')  # Exclude top layers
base_model.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [43]:

model = Sequential([
    base_model,
    Flatten(),
    BatchNormalization(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 output classes: Human, No Human
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [44]:
batch_size = 32
epochs = 10
history = model.fit(
    datagen.flow(np.array(X_train), y_train, batch_size=batch_size),
    steps_per_epoch=len(X_train) // batch_size,  # Integer division for steps per epoch
    epochs=epochs,
    validation_data=(np.array(X_test), y_test)
)

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.6589 - loss: 3.1721 - val_accuracy: 0.7081 - val_loss: 7.7538
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.7113 - loss: 4.2982 - val_accuracy: 0.7135 - val_loss: 5.6144
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.7328 - loss: 3.2284 - val_accuracy: 0.7405 - val_loss: 3.6404
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.7294 - loss: 1.7286 - val_accuracy: 0.7405 - val_loss: 1.8115
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.7579 - loss: 1.0385 - val_accuracy: 0.7568 - val_loss: 1.0623
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.7695 - loss: 0.9541 - val_accuracy: 0.7514 - val_loss: 0.9045
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.7710 - loss: 0.7897 - val_accuracy: 0.7784 - val_loss: 0.6085
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 85s 2s/step - accuracy: 0.7666 - loss: 0.8493 - val_accuracy: 0.7784 - val_loss:

In [45]:
# Predict on the test set
predictions = model.predict(np.array(X_test))

6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step


In [46]:
predicted_classes = np.argmax(predictions, axis=1)

In [47]:
y_test_classes = np.argmax(y_test, axis=1)

In [48]:

# Compare predicted classes with actual classes
correct_predictions = np.sum(predicted_classes == y_test_classes)
total_predictions = len(y_test_classes)


In [49]:
print(f"Correct Predictions: {correct_predictions}")
print(f"Total Predictions: {total_predictions}")
print(f"Accuracy on test set: {correct_predictions / total_predictions * 100:.2f}%")

Correct Predictions: 140
Total Predictions: 185
Accuracy on test set: 75.68%


In [50]:
import requests
from io import BytesIO
from PIL import Image

In [58]:
from PIL import Image

image_path = "./1.jpg"

img = Image.open(image_path)

img.show()


In [59]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

img = img.resize((224, 224))
img_array = np.array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

In [60]:
predictions = model.predict(img_array)

print("Prediction: ", predictions)
class_labels = ['No Human', 'Human']
predicted_class = class_labels[np.argmax(predictions)]

print(f"The model predicts: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Prediction:  [[5.1403166e-09 1.0000000e+00]]
The model predicts: Human


In [62]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

Saved artifact at '/tmp/tmpavt_6rfd'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_331')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  134771346160720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134771342525840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134771342526224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134771342526032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134771342524496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134771342527376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134771342527760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134771342528144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134771342527952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134771342525264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1347713425

In [63]:
with open('human_detection_model.tflite', 'wb') as f:
    f.write(tflite_model)